In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import pandas as pd

In [26]:
url = "https://raw.githubusercontent.com/patilpranjal/Auto-Grader-for-short-answer-Question-using-Siamese-Neural-Network/master/Dataset.csv"
df = pd.read_csv(
    url,
    index_col= 0
)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4925 entries, 0 to 4924
Data columns (total 5 columns):
studentAnswer      4925 non-null object
referenceAnswer    4925 non-null object
ref_1              4925 non-null object
ref_2              4925 non-null object
accuracy           4925 non-null object
dtypes: object(5)
memory usage: 230.9+ KB


In [28]:
lst = list(df.referenceAnswer.value_counts().index)

In [29]:
temp = pd.DataFrame(lst, columns = ["answer"])

In [30]:
temp['group'] = temp.index

In [31]:
df = pd.merge(
    left = df,
    left_on = "referenceAnswer",
    right = temp,
    right_on = 'answer'
)

In [32]:
df = df.drop('answer', axis = 1)

In [75]:
documents = df[df['group'] == 12].studentAnswer.values
label = df[df['group'] == 12].accuracy.values

In [76]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

In [77]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
y_pred = model.predict(X)

In [78]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind]),
    print()

Top terms per cluster:
Cluster 0:
 switch
 circuit
 complete
 motor
 work

Cluster 1:
 circle
 happen
 flowing
 battery
 strong



In [79]:
valid = pd.DataFrame(zip(documents,label, y_pred), columns= ["answers","label","group"])

In [81]:
valid[valid.label == "incorrect"]

,answers,label,group
6,It is not complete.,incorrect,0
9,Same as the light but it is switch to the othe...,incorrect,0
10,Because the light bulb is on the right.,incorrect,0
19,The switch will be off.,incorrect,0
20,It will happen because the switch moved so it ...,incorrect,0
23,Because the energy still has a countless form ...,incorrect,0
25,The switch is closer.,incorrect,0
27,The switch is not turned to the left.,incorrect,0
28,That will happen because there is no battery.,incorrect,1
29,Because the D-cell is not in the middle.,incorrect,0


In [ ]:
# Need to make elbow graph showing how errors are minimized
# Need to do some type of count that tells me how good it did in guessing